In [1]:
import cv2
import numpy as np
import pandas as pd
from scipy.fftpack import dct
from skimage.feature import local_binary_pattern
import os
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
import random
from sklearn.preprocessing import MinMaxScaler
import tqdm

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
!pwd

/home/ap6624/CV


In [4]:
split_size = 16

P = 8
R = 1.0

In [5]:

def extract_features(path_to_folder, class_label):
    label = []
    feature_vectors = []
    for file_name in tqdm.tqdm(os.listdir(path_to_folder)):
        path_to_img = os.path.join(path_to_folder,file_name)
        img = cv2.imread(path_to_img)
        if np.shape(img) == ():
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb) 
        img = img[:,:,1] 
        img_h, img_w = img.shape
        
        i = 1
        X_points = [0]
        while True:
            pt = split_size * i
            if pt + split_size >= img_w:
                pt = img_w - split_size
                X_points.append(pt)
                break
            X_points.append(pt)
            i += 1
        i = 1
        
        Y_points = [0]
        while True:
            pt = split_size * i
            if pt + split_size >= img_h:
                pt = img_h - split_size
                Y_points.append(pt)
                break
            Y_points.append(pt)
            i += 1
        
        dct_blocks=[]
        for i in X_points:
            for j in Y_points:
                block = img[i:i+split_size, j:j+split_size]
                dct_block = dct(local_binary_pattern(block, 8, 1, method='default'))
                dct_blocks.append(dct_block)

        np_dct_blocks=np.asarray(dct_blocks)
        
        std_features=[]
        
        for x in range(np_dct_blocks.shape[1]):
            for y in range(np_dct_blocks.shape[2]):
                std_features.append(np.std(np_dct_blocks[:,x,y]))
        
        feature_vectors.append(img_std_list)
        label.append(class_label)

In [6]:
labels=[]
dataframe_list=[]


truepath="/scratch/ap6624/CV/CASIA2/Au"
fakepath="/scratch/ap6624/CV/CASIA2/Tp"
truefeatures, truelabels = extract_features(truepath, 0)
fakefeatures, fakelabels = extract_features(fakepath, 1)


100%|██████████| 5124/5124 [07:40<00:00, 11.12it/s]


In [7]:
print("Length/Dimension of features", len(feature_vector[0]))
print("Length of feature vector", len(feature_vector))
print("length of label",len(label))

Length/Dimension of features 256
Length of feature vector 12614
length of label 12614


In [8]:
df=pd.DataFrame(dataframe_list)
df.rename(columns = {0: "image_names"}, inplace = True)

df.head()

,image_names,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,256,257
0,Au_art_30071.jpg,0,1567.715904,766.830652,624.437346,602.170676,439.412059,321.294301,279.062546,269.060390,...,40.551594,36.077446,35.466951,34.455235,38.219689,32.284899,31.733224,31.299188,29.615672,29.221101
1,Au_ani_30378.jpg,0,1763.982651,737.091989,653.245452,432.978036,361.855748,318.396526,296.688332,278.954918,...,37.419802,34.128734,35.515252,36.079671,38.935790,36.974607,42.711979,42.538353,43.471632,47.587055
2,Au_nat_20042.jpg,0,1077.708468,405.326826,330.520953,289.522949,274.415245,259.199431,282.642769,252.076031,...,35.884322,35.097393,33.219224,36.253793,41.359594,40.336380,36.703289,38.354853,40.352980,58.453324
3,Au_cha_30123.jpg,0,1603.514027,635.466161,475.340397,380.701089,299.040544,261.374984,261.276790,255.627437,...,35.885818,35.697738,33.811802,34.763363,36.674487,40.926432,41.100644,38.740962,42.406346,48.732615
4,Au_cha_30256.jpg,0,1594.976705,591.719444,527.464070,374.902148,314.706195,287.745363,292.904765,257.886198,...,37.111118,34.067409,37.752055,38.575956,40.769376,43.994439,41.361549,48.019603,45.565972,56.248675


In [9]:
scaler_norm = MinMaxScaler() 
df.iloc[:,1:] = scaler_norm.fit_transform(df.iloc[:,1:].to_numpy()) 

In [10]:
path_csv="CASIA2_feature_DCT_LBP.csv"

df.to_csv(path_csv)

In [11]:
array=df.values
x_feature=array[:,2:]
y_label=array[:,1].astype('int')
print(x_feature.shape)
print(y_label.shape)

X_train,X_test,Y_train,Y_test=train_test_split(x_feature,y_label,test_size=0.20,random_state=7)
model_SVC=SVC(C=32,kernel='rbf',gamma=0.03125)

kfold=KFold(n_splits=10)
cv_results=cross_val_score(model_SVC,X_train,Y_train,cv=kfold,scoring='accuracy')
msg="%s %f (%f)" % ('Training Accuracy: ',cv_results.mean(),cv_results.std())
print(msg)

(12614, 256)
(12614,)
Training Accuracy:  0.971163 (0.007769)


In [12]:
model_SVC = SVC(C=32,gamma=0.03125, kernel='rbf') 
model_SVC.fit(X_train,Y_train) 

predictions=model_SVC.predict(X_test)

print(accuracy_score(Y_test,predictions))
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

0.9659135949266746
[[1454   77]
 [   9  983]]
              precision    recall  f1-score   support

           0       0.99      0.95      0.97      1531
           1       0.93      0.99      0.96       992

    accuracy                           0.97      2523
   macro avg       0.96      0.97      0.96      2523
weighted avg       0.97      0.97      0.97      2523

